In [1]:
# --- Auto-load latest saved model from black_box_model/, then run SHAP ---
import os, glob, joblib, numpy as np, pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import shap

# -------- Paths --------
MODEL_GLOB = "black_box_model/*.joblib"
GRAPHS_DIR = "graphs"
os.makedirs(GRAPHS_DIR, exist_ok=True)

def latest_file(pattern: str) -> str:
    files = glob.glob(pattern)
    if not files:
        raise FileNotFoundError(f"No files match: {pattern}")
    return max(files, key=os.path.getmtime)

model_path = latest_file(MODEL_GLOB)
print(f"[LOAD] Using model: {model_path}")
full_pipeline = /Users/nehasharma/fair_ai_exploration/black_box_model/grad_boost_model_0925_2243.joblib

# -------- Bring X_test_df / y_test into scope --------
# Use existing vars if they already exist; otherwise rebuild a split (same seed/stratify).
try:
    X_test_df, y_test
    print("[DATA] Using in-memory X_test_df / y_test.")
except NameError:
    # Try to locate your dataset automatically
    candidates = [
        "data/dataproject2025.csv", "../data/dataproject2025.csv",
        "data/dataproject2025.xlsx", "../data/dataproject2025.xlsx",
    ]
    DATA_PATH = next((p for p in candidates if os.path.exists(p)), None)
    if DATA_PATH is None:
        raise FileNotFoundError("Could not find data file (tried CSV/XLSX in data/ and ../data/).")
    print(f"[DATA] Loading: {DATA_PATH}")
    if DATA_PATH.lower().endswith(".xlsx"):
        df = pd.read_excel(DATA_PATH)
    else:
        df = pd.read_csv(DATA_PATH)

    # Columns to drop (leakage/IDs) — adjust if needed
    ID_LEAK_COLS = ["Unnamed: 0", "Predicted probabilities", "Predictions"]
    TARGET_COLUMN = "target"

    if TARGET_COLUMN not in df.columns:
        raise KeyError(f"Target column '{TARGET_COLUMN}' not in data. Columns: {list(df.columns)[:10]} ...")

    y = df[TARGET_COLUMN].astype(int)
    X = df.drop(columns=[c for c in ID_LEAK_COLS if c in df.columns], errors="ignore")

    # Ensure we do not include the target in X
    if TARGET_COLUMN in X.columns:
        X = X.drop(columns=[TARGET_COLUMN])

    X_train_df, X_test_df, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    print(f"[DATA] Split: train={len(y_train)}, test={len(y_test)}")

# -------- SHAP on pipeline --------
pre = full_pipeline.named_steps["preprocessor"]
model = full_pipeline.named_steps["classifier"]

X_test_trans = pre.transform(X_test_df)

# Feature names after preprocessing
try:
    feature_names = pre.get_feature_names_out()
except Exception:
    # Fallback: try to reconstruct names
    feature_names = np.array([f"f{i}" for i in range(X_test_trans.shape[1])])

timestamp = datetime.now().strftime("%m%d_%H%M")

# Prefer TreeExplainer for tree-based models; fallback to model-agnostic Explainer
try:
    explainer = shap.TreeExplainer(model)
    shap_vals = explainer.shap_values(X_test_trans)
    if isinstance(shap_vals, list):  # binary class list
        shap_pos = shap_vals[1]
        expected_value = explainer.expected_value[1]
    else:
        shap_pos = shap_vals
        expected_value = explainer.expected_value
except Exception as e:
    print("[SHAP] TreeExplainer failed, using model-agnostic Explainer:", e)
    f = lambda M: model.predict_proba(M)[:, 1]
    bg = shap.sample(X_test_trans, 100, random_state=42)
    explainer = shap.Explainer(f, bg)
    exp = explainer(X_test_trans)
    shap_pos = exp.values
    expected_value = np.mean(exp.base_values)

# Global plots
plt.figure()
shap.summary_plot(shap_pos, X_test_trans, feature_names=feature_names, show=False)
beeswarm_path = os.path.join(GRAPHS_DIR, f"shap_beeswarm_{timestamp}.png")
plt.tight_layout(); plt.savefig(beeswarm_path, dpi=150); plt.close()
print(f"[SHAP] Saved beeswarm: {beeswarm_path}")

plt.figure()
shap.summary_plot(shap_pos, X_test_trans, feature_names=feature_names, plot_type="bar", show=False)
bar_path = os.path.join(GRAPHS_DIR, f"shap_importance_bar_{timestamp}.png")
plt.tight_layout(); plt.savefig(bar_path, dpi=150); plt.close()
print(f"[SHAP] Saved bar plot: {bar_path}")

# Local force plots — pick one default and one non-default
pos_indices = np.where(y_test.values == 1)[0]
neg_indices = np.where(y_test.values == 0)[0]
pos_idx = int(pos_indices[0]) if len(pos_indices) else 0
neg_idx = int(neg_indices[0]) if len(neg_indices) else 0

pos_html = os.path.join(GRAPHS_DIR, f"shap_force_pos_{timestamp}.html")
neg_html = os.path.join(GRAPHS_DIR, f"shap_force_neg_{timestamp}.html")

try:
    shap.initjs()
    pos_plot = shap.force_plot(expected_value, shap_pos[pos_idx], X_test_trans[pos_idx,:], feature_names=feature_names)
    neg_plot = shap.force_plot(expected_value, shap_pos[neg_idx], X_test_trans[neg_idx,:], feature_names=feature_names)
    shap.save_html(pos_html, pos_plot); shap.save_html(neg_html, neg_plot)
    print(f"[SHAP] Saved force plots: {pos_html} | {neg_html}")
except Exception as e:
    print("[SHAP] force_plot save_html failed; trying new API:", e)
    try:
        epos = shap.Explanation(values=shap_pos[pos_idx], base_values=expected_value,
                                data=X_test_trans[pos_idx,:], feature_names=feature_names)
        eneg = shap.Explanation(values=shap_pos[neg_idx], base_values=expected_value,
                                data=X_test_trans[neg_idx,:], feature_names=feature_names)
        shap.save_html(pos_html, shap.plots.force(epos))
        shap.save_html(neg_html, shap.plots.force(eneg))
        print(f"[SHAP] Saved force plots: {pos_html} | {neg_html}")
    except Exception as e2:
        print("[SHAP] New API also failed → printing top contributors.", e2)
        def top_k(values, names, k=10):
            idx = np.argsort(np.abs(values))[::-1][:k]
            return [(names[i], float(values[i])) for i in idx]
        print("Top-10 (positive case):", top_k(shap_pos[pos_idx], feature_names))
        print("Top-10 (negative case):", top_k(shap_pos[neg_idx], feature_names))

# Quick text: Top-10 global drivers
mean_abs = np.abs(shap_pos).mean(axis=0)
order = np.argsort(mean_abs)[::-1]
top10 = [(feature_names[i], float(mean_abs[i])) for i in order[:10]]
print("\n[SHAP] Top-10 global drivers by mean(|SHAP|):")
for name, val in top10:
    print(f" - {name}: {val:.6f}")


FileNotFoundError: No files match: black_box_model/*.joblib